In [ ]:
from utils import *

# perform QC on sequencing library using fastqc/multiqc
run_fastqc(in_dir='../01-INPUT/02-FASTQ/*',out_dir='../02-OUTPUT/01-QC/')
run_multiqc(in_dir='../02-OUTPUT/01-QC/',out_dir='../02-OUTPUT/01-QC/')

# trim sequencing reads using trimmomatic
adapter_file = "~/anaconda3/pkgs/trimmomatic-0.39-hdfd78af_2/share/trimmomatic-0.39-2/adapters/NexteraPE-PE.fa"
run_trimmomatic(in_dir="../01-INPUT/02-FASTQ",
                out_dir="../02-OUTPUT/02-TRIM",
                params=f"ILLUMINACLIP:{adapter_file}:2:30:10 LEADING:3 TRAILING:3 SLIDINGWINDOW:4:30 MINLEN:20")

# align sequencing reads to reference using bowtie2
build_bowtie2_index(in_file="../01-INPUT/03-REFERENCE/Gent_RK2.fasta",
                    out_file="../02-OUTPUT/03-ALIGN/Gent_RK2")
run_bowtie2_alignment(in_dir="../02-OUTPUT/02-TRIM",
                      out_dir="../02-OUTPUT/03-ALIGN")

# analyze sequencing results using pysamstats
run_pysamstats(in_dir="../02-OUTPUT/03-ALIGN",
               out_dir="../02-OUTPUT/04-ANALYZE")

# plot sequencing results
df = pd.read_table("../02-OUTPUT/04-ANALYZE/LBspec-mut1_S19_L001_mapping.sorted.bam.var.txt")
df = df.query("pos>3500 and pos<5500")

fig, ax = plt.subplots(2,1)
sns.lineplot(data=df,x='pos',y='mismatches',color='red',ax=ax[0])
sns.lineplot(data=df,x='pos',y='matches',color='black',ax=ax[1])